In [1]:
import pandas as pd

movies = pd.read_csv("data/ratings_movies.csv")
display(movies.head())

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [2]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [3]:
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?

# Создаем новый столбец year_release с помощью apply и функции get_year_release
movies['year_release'] = movies['title'].apply(get_year_release)

# Считаем количество фильмов, у которых не указан год выпуска
movies_without_year = movies[movies['year_release'].isnull()]
num_movies_without_year = len(movies_without_year)

print(f"Количество фильмов без указания года выпуска: {num_movies_without_year}")

Количество фильмов без указания года выпуска: 18


In [4]:
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.

# Выбор фильмов, выпущенных в 1999 году
movies_1999 = movies[movies['year_release'] == 1999]

# Группировка по movieId и вычисление средней оценки для каждого фильма
average_ratings = movies_1999.groupby('movieId')['rating'].mean().reset_index()

# Находим movieId с наименьшей средней оценкой
min_avg_rating_movieId = average_ratings.loc[average_ratings['rating'].idxmin()]['movieId']

# Находим название фильма с наименьшей средней оценкой
movie_title = movies[movies['movieId'] == min_avg_rating_movieId]['title'].values[0]

# Выводим название фильма без указания года его выпуска
print(movie_title.split(' (')[0])

Bloodsport: The Dark Kumite


In [5]:
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

# Выбор фильмов, выпущенных в 2010 году
movies_2010 = movies[movies['year_release'] == 2010]

# Группировка по жанрам (genres) и вычисление средней оценки для каждого сочетания жанров
average_ratings_genres = movies_2010.groupby('genres')['rating'].mean().reset_index()

# Находим сочетание жанров с наименьшей средней оценкой
min_avg_rating_genres = average_ratings_genres.loc[average_ratings_genres['rating'].idxmin()]['genres']

# Выводим сочетание жанров с наименьшей средней оценкой
print(min_avg_rating_genres)

Action|Sci-Fi


In [6]:
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов?
# В качестве ответа запишите идентификатор этого пользователя.

# Подсчет уникальных комбинаций жанров для каждого пользователя
unique_genres_count = movies.groupby('userId')['genres'].nunique().reset_index()

# Находим пользователя с наибольшим количеством уникальных комбинаций жанров
user_with_max_unique_genres = unique_genres_count.loc[unique_genres_count['genres'].idxmax()]['userId']

# Выводим идентификатор пользователя с наибольшим количеством уникальных комбинаций жанров
print(user_with_max_unique_genres)

599


In [8]:
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя. Чтобы рассчитать несколько параметров для каждого
# пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

# Группировка данных по userId и вычисление количества оценок и средней оценки для каждого пользователя
user_ratings_info = movies.groupby('userId').agg({'rating': ['count', 'mean']}).reset_index()

# Находим пользователя с самой высокой средней оценкой
user_highest_mean_rating = user_ratings_info[user_ratings_info[('rating', 'mean')] == user_ratings_info[('rating', 'mean')].max()]

# Находим пользователя среди тех, у кого самая высокая средняя оценка, но с наименьшим количеством оценок
user_min_ratings_max_mean = user_highest_mean_rating[user_highest_mean_rating[('rating', 'count')] == user_highest_mean_rating[('rating', 'count')].min()]

# Выводим идентификатор пользователя
print(user_min_ratings_max_mean['userId'].values[0])

53


In [9]:
# Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating),
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

# Выбор фильмов, выпущенных в 2018 году
movies_2018 = movies[movies['year_release'] == 2018]

# Группировка по жанрам (genres) и вычисление среднего рейтинга и числа оценок для каждого сочетания жанров
genres_ratings_info = movies_2018.groupby('genres')['rating'].agg(['mean', 'count']).reset_index()

# Фильтрация данных по условиям: число оценок больше 10 и нахождение сочетания жанров с наибольшим средним рейтингом
max_rating_genres = genres_ratings_info[(genres_ratings_info['count'] > 10) & (genres_ratings_info['mean'] == genres_ratings_info[genres_ratings_info['count'] > 10]['mean'].max())]['genres'].values[0]

# Вывод сочетания жанров
print(max_rating_genres)

Action|Adventure|Sci-Fi


In [10]:
# обавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует
# зависимость среднего рейтинга фильма от года выставления оценки и жанра. 
# Выберите верные варианты ответа, исходя из построенной таблицы:

# Добавление признака year_rating - год выставления оценки
movies['year_rating'] = pd.to_datetime(movies['date']).dt.year

# Создание сводной таблицы
pivot_table = movies.pivot_table(values='rating', index='genres', columns='year_rating', aggfunc='mean')

# Анализ данных по утверждениям
print("A:", pivot_table.loc['Action|Adventure'].min() >= 3)
print("B:", pivot_table.loc['Action|Adventure|Animation|Children|Comedy|IMAX', 2010] == pivot_table.loc['Action|Adventure|Animation|Children|Comedy|IMAX'].max())
print("C:", 'Animation|Children|Mystery' in pivot_table[pivot_table.index.isin(['Animation|Children|Mystery'])][2018])
print("D:", pivot_table.loc['Comedy'].idxmax() == 1996)  # Проверяем, возрастает ли рейтинг

A: True
B: False
C: True
D: False


In [14]:
# Новая задача в двумя новыми таблицами, стандартный вид чтения не сработал из-за разделителя ";"
orders = pd.read_csv("data/orders.csv", delimiter=";")
products = pd.read_csv("data/products.csv",delimiter=";")
display(orders.head())
display(products.head())

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


In [16]:
# Объединение таблиц с помощью merge, чтобы в результирующей таблице оказалась инфа обо всех заказах, необходим
# параметр how = left, это позволит сохранить все записи из таблицы orders и добавить всю соответствующую инфу из products

# Объединение таблиц по столбцу 'ID товара' с сохранением всех записей из таблицы orders
orders_products = orders.merge(products, left_on='ID товара', right_on='Product_ID', how='left')
display(orders_products.head())


,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR


In [19]:
# Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?

# Находим Order ID, для которого нет информации о товаре (NaN в столбце Product_ID)
missing_product_order = orders_products.loc[orders_products['Product_ID'].isnull(), 'Order ID'].unique()
print(missing_product_order)

[0]


In [20]:
# На какой товар была произведена отмена? В качестве ответа запишите название этого товара (Name).

# Находим название товара, для которого была отмена заказа
canceled_product_name = orders_products.loc[orders_products['Статус'] == 'Отменён', 'Name'].iloc[0]

# Выводим название отмененного товара
print(canceled_product_name)

Носки беговые Camino


In [21]:
# Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
# В ответ запишите идентификатор этого покупателя (ID Покупателя).
# Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.

# Создаем новый столбец 'TotalProfit' в таблице 'orders_products'
orders_products['TotalProfit'] = orders_products['Price'] * orders_products['Количество']

# Группируем данные по 'ID Покупателя' и суммируем значения 'TotalProfit' для каждого покупателя
customer_total_profit = orders_products.groupby('ID Покупателя')['TotalProfit'].sum()

# Находим покупателя с наибольшей суммарной прибылью
customer_max_profit = customer_total_profit.idxmax()

# Выводим идентификатор покупателя с наибольшей суммарной прибылью
print(customer_max_profit)

7
